In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/insurance/insurance.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

data=pd.read_csv('/kaggle/input/insurance/insurance.csv')
data.replace({'female':1,'male':0,'yes':1,'no':0,'southwest':1,'southeast':2,'northwest':3,'northeast':4},inplace=True)
y = data.charges
X = data.drop(['charges'], axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3,random_state=0)
print(data.head(10))
s=pd.Series()
def charges_dataset(max_leaf_nodes,X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes,n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)
for max_leaf_nodes in range(50,500,50):
    my=charges_dataset(max_leaf_nodes,X_train, X_valid, y_train, y_valid)
    s[my]=max_leaf_nodes
best_tree_size =s[s.index.min()]
print('ERROR -',charges_dataset(best_tree_size,X_train, X_valid, y_train, y_valid))
best_model = RandomForestRegressor(max_leaf_nodes=best_tree_size,n_estimators=100, random_state=0)
best_model.fit(X,y)

/tmp/ipykernel_30/654108522.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({'female':1,'male':0,'yes':1,'no':0,'southwest':1,'southeast':2,'northwest':3,'northeast':4},inplace=True)


   age  sex     bmi  children  smoker  region      charges
0   19    1  27.900         0       1       1  16884.92400
1   18    0  33.770         1       0       2   1725.55230
2   28    0  33.000         3       0       2   4449.46200
3   33    0  22.705         0       0       3  21984.47061
4   32    0  28.880         0       0       3   3866.85520
5   31    1  25.740         0       0       2   3756.62160
6   46    1  33.440         1       0       2   8240.58960
7   37    1  27.740         3       0       3   7281.50560
8   37    0  29.830         2       0       4   6406.41070
9   60    1  25.840         0       0       3  28923.13692
ERROR - 2640.777984740698


RandomForestRegressor(max_leaf_nodes=50, random_state=0)

In [3]:
test=pd.DataFrame({'age':[71,19,49],'sex':[0,1,0],'bmi':[28.4047635997,17.96875,25.5990428825],'children':[4,0,3],'smoker':[0,0,1],'region':[4,4,3]})
test_preds = best_model.predict(test)
test['charges']=test_preds
test.sex.replace({1:'female',0:'male'},inplace=True)
test.smoker.replace({1:'yes',0:'no'},inplace=True)
test.region.replace({1:'southwest',2:'southeast',3:'northwest',4:'northeast'},inplace=True)
print(test)

   age     sex        bmi  children smoker     region       charges
0   71    male  28.404764         4     no  northeast  24890.166145
1   19  female  17.968750         0     no  northeast   2611.438561
2   49    male  25.599043         3    yes  northwest  23754.323326


/tmp/ipykernel_30/3161971184.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test.sex.replace({1:'female',0:'male'},inplace=True)
/tmp/ipykernel_30/3161971184.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 